# MPI example - Torchfort

2025-06-05

In [14]:
TFT="~/containers/torchfort.sif"

In [9]:
! singularity inspect {TFT}

maintainer: NVIDIA CORPORATION <cudatools@nvidia.com>
org.label-schema.build-arch: amd64
org.label-schema.build-date: Tuesday_27_May_2025_16:26:52_-03
org.label-schema.schema-version: 1.0
org.label-schema.usage.singularity.deffile.bootstrap: docker-daemon
org.label-schema.usage.singularity.deffile.from: torchfort:latest
org.label-schema.usage.singularity.version: 4.3.1-jammy
org.opencontainers.image.ref.name: ubuntu
org.opencontainers.image.version: 22.04


In [10]:
! singularity exec --nv {TFT} mpiexec --version

mpiexec (OpenRTE) 4.1.7a1

Report bugs to http://www.open-mpi.org/community/help/


In [11]:
! singularity exec --nv {TFT} mpif90 --version


nvfortran 24.1-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2024, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


In [12]:
! rm sum_mpi

In [13]:
! singularity exec --nv {TFT} mpif90 -o sum_mpi sum_mpi.f90

In [15]:
! singularity exec --nv {TFT} mpirun -np 4 ./sum_mpi

 Process             2  (of             4 ) has local_sum =            78
 Process             0  (of             4 ) has local_sum =            55
 Process             3  (of             4 ) has local_sum =            91
 Process             1  (of             4 ) has local_sum =            66
 ----------------------------------
 Global sum calculated by root (rank 0):           290
 ----------------------------------


In [18]:
! cp sum_mpi /scratch${HOME#/prj}

In [20]:
! cp ~/containers/torchfort.sif /scratch${HOME#/prj}

In [19]:
! ls /scratch${HOME#/prj}

sum_mpi


Queues:
https://github.com/lncc-sered/manual-sdumont/wiki/06-%E2%80%90--Gerenciador-de-filas

    Fila    sequana_cpu_dev
    Wall-clock máximo (em horas)    0:20
    Número mínimo de nós (núcleos+ dispositivos)    1 (48)
    Número máximo de nós (núcleos+ dispositivos)    4 (192) 
    Número máximo de tarefas em execução por usuário    1
    Número máximo de tarefas em fila por usuário    1
    Custo em Unidade de Alocação (UA)   1,0
    
    Fila 	sequana_gpu_dev
    Wall-clock máximo (em horas) 	0:20
    Número mínimo de nós (núcleos+ dispositivos) 	1 (48+4)
    Número máximo de nós (núcleos+ dispositivos) 	4 (192+16)
    Número máximo de tarefas em execução por usuário 	1
    Número máximo de tarefas em fila por usuário 	1
    Custo em Unidade de Alocação (UA)   1,5

In [46]:
! module show openmpi/gnu/4.1.4_sequana

----------------------------------------------------------------------------
   /petrobr/app_sequana/modulos2/openmpi/gnu/4.1.4_sequana:
----------------------------------------------------------------------------
whatis("Sets up Open MPI/SHMEM v4.1.4, compiled with GCC 9.3 and with UCX 1.13.0
, in your enviornment")
prepend_path("PATH","/scratch/app/openmpi/4.1.4_gnu/bin")
prepend_path("MANPATH","/scratch/app/openmpi/4.1.4_gnu/share/man")
prepend_path("LD_LIBRARY_PATH","/scratch/app/openmpi/4.1.4_gnu/lib:/usr/lib64/ps
m2-compat")
prepend_path("PKG_CONFIG_PATH","/scratch/app/openmpi/4.1.4_gnu/lib/pkgconfig")
setenv("MPI_ROOT","/scratch/app/openmpi/4.1.4_gnu")
setenv("OMPI_PATH","/scratch/app/openmpi/4.1.4_gnu/bin")
setenv("OMPI_LD_LIBRARY_PATH","/scratch/app/openmpi/4.1.4_gnu/lib")
setenv("OMPI_MCA_btl_openib_allow_ib","1")
setenv("OMPI_MCA_btl_openib_warn_default_gid_prefix","0")
setenv("OMPI_MCA_btl_vader_single_copy_mechanism","2")
prepend_path{"LDFLAGS","-L/scratch/app/openmpi/4.1.

In [57]:
%%writefile sum_mpi.srm
#!/bin/bash
#SBATCH --partition=sequana_cpu_dev   # SLURM_JOB_PARTITION
#SBATCH --job-name=hv-tf1-mnist       # SLURM_JOB_NAME
#SBATCH --nodes=4                     # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=1           # SLURM_NTASKS_PER_NODE
#SBATCH --cpus-per-task=1             # SLURM_CPUS_PER_TASK
#SBATCH --distribution=block:cyclic   # SLURM_DISTRIBUTION
#SBATCH --time=00:05:00               # Limit execution time
# SLURM output environment variables:
# https://slurm.schedmd.com/sbatch.html#SECTION_OUTPUT-ENVIRONMENT-VARIABLES
echo '======================================='
echo '- Job ID:' $SLURM_JOB_ID
echo '- Nº of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- Nº of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Nº of tasks:' $SLURM_NTASKS
echo '- Nº of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Partition:' $SLURM_JOB_PARTITION
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST
echo '----------------------------------------'
#cd $SLURM_SUBMIT_DIR
cd /scratch/ampemi/eduardo.miranda2/
# Run
export SLURM_MPI_TYPE=pmi2
#export OMPI_HOME=$(dirname $(dirname $(which mpirun)))
module load openmpi/gnu/4.1.4+cuda-11.2_sequana
echo -n '<1. starting python script > ' && date
echo '-- output -----------------------------'
#srun --cpu_bind=cores singularity exec torchfort.sif mpirun -np 4 ./sum_mpi
#srun --mpi=pmi2 -n 4 hostname
#srun singularity exec \
#    --bind /usr/lib64/mpi:/usr/lib64/mpi \
#    torchfort.sif ./sum_mpi
#srun --mpi=none singularity exec torchfort.sif mpirun --allow-run-as-root ./sum_mpi
srun singularity exec --nv --bind /usr/lib64:/host_libs \
    torchfort.sif bash -c "export LD_LIBRARY_PATH=/host_libs:\$LD_LIBRARY_PATH && ./sum_mpi"
echo '-- end --------------------------------'
echo -n '<2. quit>                    ' && date

Overwriting sum_mpi.srm


In [58]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342271
- Job ID: 11342271
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6078 sdumont6165 sdumont6166 sdumont6167
----------------------------------------
<1. starting python script > qui jun  5 23:41:40 -03 2025
-- output -----------------------------
./sum_mpi: /host_libs/libc.so.6: version `GLIBC_2.34' not found (required by ./sum_mpi)
./sum_mpi: /host_libs/libc.so.6: version `GLIBC_2.34' not found (required by ./sum_mpi)
./sum_mpi: /host_libs/libc.so.6: version `GLIBC_2.34' not found (required by ./sum_mpi)
srun: error: sdumont6167: task 3: Exited with exit code 1
srun: error: sdumont6166: task 2: Exited with exit code 1
srun: error: sdumont6165: task 1: Exited with exit code 1
./sum_mpi: /host_libs/libc.so.6: version `GLIBC_2.34' not found (required by ./sum_mpi)
srun: error: sdumont

In [36]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342253
- Job ID: 11342253
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6165 sdumont6166 sdumont6167
----------------------------------------
<1. starting python script > qui jun  5 23:17:44 -03 2025
-- output -----------------------------
srun: defined options
srun: -------------------- --------------------
srun: (null)              : sdumont[6075,6165-6167]
srun: cpus-per-task       : 1
srun: distribution        : block:cyclic
srun: jobid               : 11342253
srun: job-name            : hv-tf1-mnist
srun: mem-per-cpu         : 8000
srun: mpi                 : pmi2
srun: nodes               : 4
srun: ntasks              : 4
srun: ntasks-per-node     : 1
srun: tres-per-task       : cpu:1
srun: verbose             : 1
srun: -------------------- --------------------
srun: e

In [34]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342250
- Job ID: 11342250
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6078 sdumont6079 sdumont6082
----------------------------------------
<1. starting python script > qui jun  5 23:16:04 -03 2025
-- output -----------------------------
srun: defined options
srun: -------------------- --------------------
srun: (null)              : sdumont[6075,6078-6079,6082]
srun: cpus-per-task       : 1
srun: distribution        : block:cyclic
srun: jobid               : 11342250
srun: job-name            : hv-tf1-mnist
srun: mem-per-cpu         : 8000
srun: mpi                 : pmi2
srun: nodes               : 4
srun: ntasks              : 4
srun: ntasks-per-node     : 1
srun: tres-per-task       : cpu:1
srun: verbose             : 1
srun: -------------------- --------------------
sr

In [28]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342241
- Job ID: 11342241
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6078 sdumont6079 sdumont6082
----------------------------------------
<1. starting python script > qui jun  5 23:09:33 -03 2025
-- output -----------------------------
--------------------------------------------------------------------------
The SLURM process starter for OpenMPI was unable to locate a
usable "srun" command in its path. Please check your path
and try again.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
An internal error has occurred in ORTE:

[[2548,0],0] FORCE-TERMINATE AT (null):1 - error ../../../../../orte/mca/plm/slurm/plm_slurm_module.c(475)

This is something that should be repor

In [26]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342239
- Job ID: 11342239
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6165 sdumont6166 sdumont6167
----------------------------------------
<1. starting python script > qui jun  5 23:08:02 -03 2025
-- output -----------------------------
--------------------------------------------------------------------------
Sorry!  You were supposed to get help about:
    opal_init:startup:internal-failure
But I couldn't open the help file:
    /proj/nv/libraries/Linux_x86_64/24.1/hpcx-12/243364-rel-1/comm_libs/12.2/hpcx/hpcx-2.17.1/ompi/share/openmpi/help-opal-runtime.txt: No such file or directory.  Sorry!
--------------------------------------------------------------------------
--------------------------------------------------------------------------
Sorry!  You were supposed to ge

In [24]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342233
- Job ID: 11342233
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6078 sdumont6079 sdumont6166 sdumont6167
----------------------------------------


In [21]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342230
- Job ID: 11342230
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6078 sdumont6079 sdumont6082
----------------------------------------
<1. starting python script > qui jun  5 22:59:49 -03 2025
-- output -----------------------------
sdumont6082
sdumont6079
sdumont6075
sdumont6078
-- end --------------------------------
<2. quit>                    qui jun  5 22:59:49 -03 2025


In [18]:
import time
a = ! sbatch sum_mpi.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=sum_mpi
    if len(b) < 2 : break
b = ! echo /scratch${PWD#/prj}/slurm-
%cat {b[0]+a[0].replace('Submitted batch job ','')}.out

Submitted batch job 11342214
- Job ID: 11342214
- Nº of nodes in the job: 4
- Nº of tasks per node: 1
- Nº of tasks: 4
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: mpi-simple-example-fortran
- Nodes allocated to the job: sdumont6075 sdumont6165 sdumont6166 sdumont6167
----------------------------------------
<1. starting python script > qui jun  5 22:31:54 -03 2025
-- output -----------------------------
--------------------------------------------------------------------------
The SLURM process starter for OpenMPI was unable to locate a
usable "srun" command in its path. Please check your path
and try again.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
An internal error has occurred in ORTE:

[[15254,0],0] FORCE-TERMINATE AT (null):1 - error ../../../../../orte/mca/plm/slurm/plm_slurm_module.c(475)

This is something that should be repo